# Der erwartungswert im allgemeinen Fall mit Hilfe des Gleichungssystems
Der Erwartungswert einer iterierten Summe von AR(1) mit dem Wort $W=i_1i_2\dots i_k$ laut Satz 7.3.33. hat der Form:
 \begin{equation}\operatorname{E}[\langle\operatorname{ISS}(X),W\rangle]=W!(\frac{\sigma^2}{1+\varphi})^{\frac{W}{2}}\sum_{\mathfrak{dw} \in \mathfrak{DW}}P(\varphi,\mathfrak{dw})\langle \operatorname{ISS}(Z),\mathfrak{dw} \rangle\end{equation}\begin{align*}\mathcal P(\varphi,\mathfrak{dw}):=\sum_{w_{\mathfrak{dw}} \in \mathfrak W_{\mathfrak{dw}}}\frac{\mathfrak{fq}_{(\mathfrak{dw},w_{\mathfrak{dw}})}}{\frac{W-w_{\mathfrak{dw}}}{2}! w_{\mathfrak{dw}}!}(\frac{- (1-\varphi)}{ \varphi})^{\frac{w_{\mathfrak{dw}}}{2}}\end{align*}\begin{equation*}
    \mathfrak{fq}_{(\mathfrak{dw},w_{\mathfrak{dw}})}=w_{\mathfrak{dw}}!\sum_{p \in\mathbf{Solutions}_{(\mathfrak{dw},w_{\mathfrak{dw}})}}\frac{1}{p!}\end{equation*}

In [1]:
import numpy as np
import pandas as pd 
import copy
import itertools as it
from itertools import product
import scipy
from scipy.special import comb
import pprint
import sympy
import math

 In der Formel des Erwartungswerts summiert man über die die Menge $\mathfrak{DW}$ die Menge der Projektionen der $\Omega_{2m}$ Dyck-Wörter der Länge $2m=i_1+i_2+\dots+i_k$, diese Menge ist in der Masterarbeit in Definition 7.3.23 und man kann sie mit Hilfe der in Bemerkung 7.3.25 definierten Funktion  finden $\mathfrak{f}_W$.
\begin{align*}
   &\mathfrak{f}_W: \Omega_{2m}\rightarrow \mathbb Z^{k}, \omega_{2m} \rightarrow z \\& z_l=\sum_{\ell =t_{l-1}}^{t_{l}} \omega_\ell \text{ with } t_{l-1}=\sum_{\ell =1}^{l-1} i_\ell \text{ and } t_{l}=\sum_{\ell =1}^{l} i_\ell
\end{align*}

Zuerst führe Der folgende Algorithmus erzeugt die Menge der Dyckwörter mithilfe der Positionen von -1 wieder ein. Diser Algorithmus produziert die Menge der Dyck-Wörter der Länge $2m$.

In [2]:
def Positions_Minuses_inDyckwords(W):
    #gernrating the dyck words with the length (2k)
    k =int(sum(W)/2)
    #the array of the positions of the 1's (the "-")
    POS=np.zeros([1,k], dtype = int)
    #the number of the perfect matching of the set {1,2k}, which ist equivalent to each Dyck Word
    Coeff=np.ones([0,1], int)
    A=np.zeros([1,k], dtype = int)
    #the Dyck words represented mit "+","-"
    Dyck_Words=np.full([0,2*k], +1)
    #the first( smallest ) word
    for i in range(0,k):
      POS[0][i]=2*(i+1)
      A[0][i]=2*(i+1)
    ind=True
    while ind == True:
        ind=False
        C=np.full([1,2*k], +1)
        B=np.ones([1,1], dtype = int)
        for i in range(0,k):
            B=B*(A[0][i]+1-2*(i+1))
            C[0][A[0][i]-1]=-1
        Coeff=np.concatenate((Coeff,B),axis=0)
        Dyck_Words = np.concatenate((Dyck_Words,C),axis=0)
        for i in range(k-2,-1,-1):
            if A[0][i]< k+1+i:
               A[0][i]=A[0][i]+1
               for j  in range(i+1,k-1):
                   A[0][j]=max(A[0][j-1]+1,2*(j+1))
               ind=True
               POS=np.concatenate((POS,A), axis = 0)
               break
    return(Dyck_Words)

 #### Die folgende Funktion findet die Projetionen der Dyck-Wörter auf das Wort $W$. Diese Projektionen sind in Definition 7.3.23. in der Masterarbeit definert.
 Mit Hilfe der Bemerkung 7.3.25. und der Funktion 
 \begin{align*} &\mathfrak{f}_W: \Omega_{2m}\rightarrow \mathbb Z^{k}, \omega_{2m} \rightarrow z \\ & z_l=\sum_{\ell =t_{l-1}}^{t_{l}} \omega_\ell \text{ with } t_{l-1}=\sum_{\ell =1}^{l-1} i_\ell \text{ and } t_{l}=\sum_{\ell =1}^{l} i_\ell\end{align*} Die folgende Funktion findet    $\mathfrak{DW}$ die Menge der Projetionen der Dyck-Wörter auf das Wort $W$.

In [3]:
def find_the_Projektions(W):
    POS=Positions_Minuses_inDyckwords(W)
    indexes=W[:-1]
    return(np.unique(np.add.reduceat(POS,np.cumsum([0,*indexes]),axis=1),axis=0))

Für ein Wort $W=i_1i_2\dots i_k $ definiere ich das Kartesische Produkt $\mathsf{PROD}=[\lfloor\frac{i_1}{2}\rfloor]\times[\lfloor\frac{i_2}{2}\rfloor]\times\dots\times[\lfloor\frac{i_k}{2}\rfloor]$. Dann gilt:
  \begin{equation}P^2_{\mathbb M[i_1i_2\dots i_k]}=\bigcup_{ \mathsf{prod}\in \mathsf{PROD} }(11)^{\mathsf{prod}_1}\star (22)^{\mathsf{prod}_2}\star\dots\star (kk)^{\mathsf{prod}_k}\star P^2_{\mathbb K_{\underbrace{W-2\mathsf{prod}}_{w}}}\end{equation}
Ich definiere die Wörtermenge der Teilgraphen, aus denen die Menge der perfekten Matchings (Paarungen) besteht:
 \begin{equation}\mathfrak W:=\{w| w= w \text{ für alle }\mathsf{prod} \in \mathsf{PROD}, (\sum_{l=1}^k w_l -2 \mathbf{ max }_l \; w_l) \ge 0\}\end{equation}
Die folgende Funktion findet die Menge $\mathfrak W$

In [4]:
def rounded_halves(A):
    X=[]
    for i in range(len(A)):
        X.append(list(range(int(np.floor(A[i]/2)+1))))
    return(X)
def Possible_Graphs(W):
    Graphs=np.zeros((0,len(W)),dtype=int)
    X=rounded_halves(W)
    cartesian_product = [i for i in it.product(*X)]
    for x in cartesian_product:
          A=np.array(x,int)
          Graph=np.array(2*A+(W%2),int)
          Diff=sum(Graph)-2*Graph[np.argmax(Graph)]
          if Diff>=0:
                Graphs=np.concatenate((Graphs,Graph.reshape(1,len(W))),axis=0)
    return(Graphs)

In der Summe steht auch das Polynom $\mathcal P(\varphi,\mathfrak{dw})$, das eine Summe über $\mathfrak W_{\mathfrak{dw}}$, die eine Teil Menge von $\mathfrak W$ ist, mit der Bedingung, \begin{align}
\text{Für ein }\mathfrak{dw}\; \forall w \in \mathfrak W \text{ mit }    \mathfrak{dw}_1=w_1,\;|\mathfrak{dw}_l|\le w_l,\; -\mathfrak{dw}_k=w_k \Rightarrow  w \in \mathfrak W_{\mathfrak{dw}}
\end{align}
Die folgende Funktion findet für jede Projektion $\mathfrak{dw}$ die Menge $\mathfrak W_{\mathfrak{dw}}$

In [5]:
def Projektions_and_its_Graphs(W):
    Graphs=Possible_Graphs(W)
    Projektions=find_the_Projektions(W)
    Graphs_of_a_Projektion=[]
    for x in Projektions:
        absolute=np.absolute(x)
        graphs_of_x=np.zeros((0,len(x)),dtype=int)
        for y in Graphs:
            if absolute[0]==y[0] and absolute[-1]==y[-1] and (absolute<=y).all():
                graphs_of_x=np.concatenate((graphs_of_x,y.reshape(1,len(y))),axis=0)
        Graphs_of_a_Projektion.append([x,graphs_of_x])
    return(Graphs_of_a_Projektion)

Nachdem man die Menge $\mathfrak W_{\mathfrak{dw}}$ für alle $\mathfrak{dw} \in \mathfrak{DW}$ brechnet hat, bleibt nur der Frequenz der Projektion eines Dyck-Wöter $\mathfrak{dw}$, um das Polynom $\mathcal P(\varphi,\mathfrak{dw})$ zu berrechnen- \begin{align*}\mathcal P(\varphi,\mathfrak{dw}):=\sum_{w_{\mathfrak{dw}} \in \mathfrak W_{\mathfrak{dw}}}\frac{\mathfrak{fq}_{(\mathfrak{dw},w_{\mathfrak{dw}})}}{\frac{W-w_{\mathfrak{dw}}}{2}! w_{\mathfrak{dw}}!}(\frac{- (1-\varphi)}{ \varphi})^{\frac{w_{\mathfrak{dw}}}{2}}\end{align*}\begin{equation*}
    \mathfrak{fq}_{(\mathfrak{dw},w_{\mathfrak{dw}})}=w_{\mathfrak{dw}}!\sum_{p \in\mathbf{Solutions}_{(\mathfrak{dw},w_{\mathfrak{dw}})}}\frac{1}{p!}\end{equation*}
um die Frequenz zu berrechnen, braucht man $\mathbf{Solutions}_{(\mathfrak{dw},w_{\mathfrak{dw}})}$ die Lösungsmenge des in der (Definition 7.3.26) präsentierte (Projektion)-(Matching)-Gleichungssystem, das ein lineares diophantisches Gleichungssystem ist und  die folgende Form hat: 
\begin{align*}&p_{12}+p_{13}+\dots+p_{1k}=w_1\\&p_{12}+p_{23}+\dots+p_{2k}=w_2\\&\qquad \vdots \\&p_{1l}+p_{2l}+\dots+p_{(l-1)l}+p_{l(l+1)}+\dots+p_{2k}=w_l\\&\qquad \vdots\\&p_{1k}+p_{2k}+\dots+p_{(k-1)k}=w_k\\&p_{12}+p_{13}+\dots+p_{1k}=\mathfrak{dw}_1\\&-p_{12}+p_{23}+\dots+p_{2k}=\mathfrak{dw}_2\\&\qquad \vdots\\&-p_{1l}-p_{2l}-\dots-p_{(l-1)l}+p_{l(l+1)}+\dots+p_{2k}=\mathfrak{dw}_l \\&\qquad \vdots \\&-p_{1k}-p_{2k}-\dots-p_{(k-1)k}=\mathfrak{dw}_k\end{align*}
Das Gleichungssystem kann in Matrixform geschrieben werden,\begin{align*}
        \mathbf{Matrix}\cdot
     \begin{pmatrix}
     p_{12}\\p_{13}\\p_{14}\\p_{23}\\ \vdots \\p_{k-1k}
     \end{pmatrix}=\begin{pmatrix}
     w^\top\\\mathfrak{dw}^\top
     \end{pmatrix}
     \end{align*}
Die folgende Funktion brechnet die Matrix des Gleichungssystems, welche nur von der Länge des Wort $W$ abhängt.

In [6]:
def Matrix_of_the_leqs_(k):
    L=list(it.combinations(range(1,k+1),2))
    M=np.zeros((2*k,len(L)),int)
    for i in range(1,k+1):
        for j in range(len(L)):
            if i == L[j][0]:
                M[i-1,j]=1
                M[k+i-1,j]=1
            elif i==L[j][1]:
                M[i-1,j]=1
                M[k+i-1,j]=-1
    return(M)

Und der folgende Code berechnet für jede $\mathfrak{dw}$ die rechte Seite des Gleichungssystems:

In [7]:
W=np.array([1,2,3,4])
Projektions_and_its_Graphs1=Projektions_and_its_Graphs(W)

In [8]:
Rightside_Equqtion_system=[]
for x in Projektions_and_its_Graphs1:
    v=[]
    for y in x[1]:
        v.append(np.concatenate((y,x[0])))
    Rightside_Equqtion_system.append([x[0],v])

In [9]:
Matrix=Matrix_of_the_leqs_(len(W))

In [10]:
Matrix

array([[ 1,  1,  1,  0,  0,  0],
       [ 1,  0,  0,  1,  1,  0],
       [ 0,  1,  0,  1,  0,  1],
       [ 0,  0,  1,  0,  1,  1],
       [ 1,  1,  1,  0,  0,  0],
       [-1,  0,  0,  1,  1,  0],
       [ 0, -1,  0, -1,  0,  1],
       [ 0,  0, -1,  0, -1, -1]])

In [11]:
Rightside_Equqtion_system

[[array([ 1,  0, -1,  0]),
  [array([ 1,  0,  1,  0,  1,  0, -1,  0]),
   array([ 1,  2,  1,  0,  1,  0, -1,  0]),
   array([ 1,  2,  3,  0,  1,  0, -1,  0])]],
 [array([ 1,  0,  1, -2]),
  [array([ 1,  0,  1,  2,  1,  0,  1, -2]),
   array([ 1,  0,  3,  2,  1,  0,  1, -2]),
   array([ 1,  2,  1,  2,  1,  0,  1, -2]),
   array([ 1,  2,  3,  2,  1,  0,  1, -2])]],
 [array([ 1,  0,  3, -4]),
  [array([ 1,  0,  3,  4,  1,  0,  3, -4]),
   array([ 1,  2,  3,  4,  1,  0,  3, -4])]],
 [array([ 1,  2, -3,  0]), [array([ 1,  2,  3,  0,  1,  2, -3,  0])]],
 [array([ 1,  2, -1, -2]),
  [array([ 1,  2,  1,  2,  1,  2, -1, -2]),
   array([ 1,  2,  3,  2,  1,  2, -1, -2])]],
 [array([ 1,  2,  1, -4]),
  [array([ 1,  2,  1,  4,  1,  2,  1, -4]),
   array([ 1,  2,  3,  4,  1,  2,  1, -4])]]]

Der notebook findet das Gleichungssystem aber löst es nicht 